In [1]:
%reload_ext cython

In [2]:
%%cython
cdef class Fate2:
    cdef long long a, b, c, r  

    def __init__(self, int seed = 1234567):
        self.a = 524_287
        self.b = 37
        self.c = 2_147_483_647
        self.r = seed

    def get_next(self):
        self.r = (self.r * self.a + self.b) % self.c
        return self.r

In [3]:
class Fate3:
    def __init__(self, seed: int = 1234567):
        self.a: int = 524_287
        self.b: int = 37
        self.c: int = 2_147_483_647
        self.r: int = seed

    def get_next(self)->int:
        self.r = (self.r * self.a + self.b) % self.c
        return self.r

In [4]:
from numba import int64
from numba.experimental import jitclass
from numba import jit

@jitclass([
  ('seed', int64),
  ('a', int64), ('b', int64), ('c', int64), ('r', int64)
])
class Fate4:
    # class members are not supported yet by numba
    def __init__(self, seed: int = 1234567):
        self.a = 524_287        # Mersenne prime
        self.b = 37
        self.c = 2_147_483_647  # Mersenne prive, found by Euler
        self.r = seed

    def get_next(self)->int64:
        self.r = (self.r * self.a + self.b) % self.c
        return self.r

In [5]:
%%cython
class Fate1:
    a: int = 524_287        # Mersenne prime
    b: int = 37
    c: int = 2_147_483_647  # Mersenne prive, found by Euler

    def __init__(self, seed: int = 1234567):
        self.r: int = seed

    def get_next(self)->int:
        self.r = (self.r * Fate1.a + Fate1.b) % Fate1.c
        return self.r

In [6]:
class Fate0:
    a = 524_287        # Mersenne prime
    b = 37
    c = 2_147_483_647  # Mersenne prive, found by Euler

    def __init__(self, seed: int = 1234567):
        self.r = seed

    def get_next(self):
        self.r = (self.r * Fate0.a + Fate0.b) % Fate0.c
        return self.r

In [7]:
import timeit

def load(fate_class):
    f = fate_class()
    for _ in range(1_000_000):
        r = f.get_next()
    return r

def test(fate_class):
    f = fate_class()
    for _ in range(1_000):
        r = f.get_next()
    return r

for fc in [Fate0, Fate1, Fate2, Fate3, Fate4]:
    print(fc, test(fc), timeit.timeit(lambda: load(fc), number=100) / 100)

<class '__main__.Fate0'> 54276433 0.4838657485500002
<class '_cython_magic_2ea1bb1b3c0f8153dad838c634010189.Fate1'> 54276433 0.4298303755800001
<class '_cython_magic_4a6d35860a4626e0e02247dd819af559.Fate2'> 54276433 0.10105763215000024
<class '__main__.Fate3'> 54276433 0.41546113848999994
<class 'numba.experimental.jitclass.base.Fate4'> 54276433 1.0314834382200002


По полученным после выполнения кода данным (которые показывают среднее время выполнения запроса на 1000 повторений) можно сделать вывод, что самым быстрым по времени выполнения запроса является класс Fate2, а самым медленным - класс Fate4. Классы Fate1 и Fate3 (а также Fate0) имеют примерно одинаковое время выполнения.